In [1]:
import pandas as pd

In [72]:
import numpy as np

In [5]:
import requests
import time
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point

## 필요한 칼럼 

#### 행정동 기본 정보 관련 
- adm_code: 행정동 shp or API 기준 고유 코드
- adm_name: 좌표 → 행정동 이름 (리버스 지오코딩)
- lat, lon: 해당 행정동의 중심좌표
- region_group: adm_name 문자열 기준 파싱 → 서울/경기/인천

#### 이동량 관련 
- move_count: 이동량

#### 충전소 관련 
- charger_station_count: groupby(adm_code) + nunique(충전소명)
- charger_total_count: 충전기 ID 수 (groupby(adm_code).size())
- charger_public_count: 공영 필터 후 count
- charger_private_count: 민영 필터 후 count

#### 공영 주차장 관련
- parking_lot_count: groupby(adm_code) + nunique(주차장명)
- parking_space_total: groupby(adm_code) + sum(구획 수)
- parking_public_count: 공영 조건 필터 후 groupby
- parking_private_count: 민영 조건 필터 후 groupby


#### 격차 및 분석 
- gap: move_count - charger_total_count
- gap_signedlog_emph: np.sign(gap) * (abs(gap) ** 1.5) 또는 signed log
- highlight: True/False


### 최종 DB 스키마

In [49]:
columns = [
    # 📍 행정 정보
    "adm_code",             # 행정동 코드 (str or int)
    "adm_name",             # 행정동 이름
    "lat", "lon",           # 중심 좌표
    "region_group",         # 서울 / 경기 / 인천
    "year",                 # 2021 / 2026 / 2030

    "move_count",           # EV 이동량 (출+도착 포함 총합)

    "charger_station_count",     # 충전소 개수
    "charger_total_count",       # 충전기 개수
    "charger_public_count",      # 공영 충전기 수
    "charger_private_count",     # 민영 충전기 수

    "parking_lot_count",         # 주차장 수
    "parking_space_total",       # 총 주차면 수
    "parking_public_count",      # 공영 주차장 수
    "parking_private_count",     # 민영 주차장 수

    "gap",                       # 이동량 - 충전기 수
    "gap_signedlog_emph",       # 강조 지표 (signed log or pow)
    "gap_rank_percentile",      # 백분위수
    "highlight"                 # 상하위 10% 여부 (True/False)
]

# 뼈대만 생성 (빈 데이터프레임)
db_schema = pd.DataFrame(columns=columns)

### 재료 DB 로드

In [21]:
#공영 주차장
pk = pd.read_csv("수도권_주차장데이터.csv")

#충전소 
ch = pd.read_csv("수도권_충전소데이터.csv")

#이동량 
d_s_21_scale = pd.read_csv("21년도_수요-공급_스케일링.csv")

d_s_26_scale = pd.read_csv("26년도_수요-공급_스케일링.csv") 

d_s_30_scale = pd.read_csv("30년도_수요-공급_스케일링.csv")

In [22]:
#head들 

print(f"주차장\n {pk.head()} \n")
print(f"충전소\n {ch.head()} \n")

print(f"21년도 수요 - 공급 스케일링\n{d_s_21_scale.head()} \n")
print(f"26년도 수요 - 공급 스케일링\n {d_s_26_scale.head()} \n")
print(f"30년도 수요 - 공급 스케일링\n {d_s_30_scale.head()} \n")

주차장
         주차장관리번호       주차장명 주차장구분 주차장유형              소재지도로명주소  \
0  122-2-000023        대청역    공영    노외   서울특별시 강남구 개포로 623-1   
1  122-2-000027  일원1동(기계식)    공영    노외  서울특별시 강남구 양재대로 27길 5   
2  122-2-000024       영희초교    공영    노외       서울특별시 강남구 일원로21   
3  122-2-000028       대왕초교    공영    노외  서울특별시 강남구 헌릉로 618길 8   
4  122-2-000026    밤고개로21길    공영    노외                   NaN   

             소재지지번주소  주차구획수 급지구분 부제시행구분        운영요일  ...       전화번호  \
0                NaN    162    4    미시행  평일+토요일+공휴일  ...  1544-3113   
1                NaN     66    4    미시행  평일+토요일+공휴일  ...  1544-3113   
2                NaN    185    4    미시행  평일+토요일+공휴일  ...  1544-3113   
3                NaN    101    4    미시행  평일+토요일+공휴일  ...  1544-3113   
4  서울특별시 강남구 율현동 529    109    4    미시행  평일+토요일+공휴일  ...  1544-3113   

         lat         lon 장애인전용주차구역보유여부     데이터기준일자   제공기관코드      제공기관명   주소  \
0  37.494949  127.079318           NaN  2024-07-05  3220000  서울특별시 강남구  NaN   
1  37.489565  127.081744 

### 스키마 적용 최종 DB 채우기

In [26]:
#admin 정보 가져오는 함수 
def get_adm_info_kakao(lat, lon, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lon, "y": lat}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result["documents"]:
                doc = result["documents"][0]
                return {
                    "adm_code": doc.get("code"),
                    "adm_name": doc.get("region_3depth_name"),
                    "region_group": doc.get("region_1depth_name")
                }
    except Exception as e:
        print("API Error:", e)
    return {"adm_code": None, "adm_name": None, "region_group": None}

In [28]:
# 🔧 Kakao API 호출 함수
def get_adm_info_kakao(lat, lon, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lon, "y": lat}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if result["documents"]:
            doc = result["documents"][0]
            return {
                "adm_code": doc.get("code"),
                "adm_name": doc.get("region_3depth_name"),
                "region_group": doc.get("region_1depth_name")
            }
    except Exception as e:
        print(f"❌ API 예외 발생: {repr(e)} | 좌표: ({lat:.5f}, {lon:.5f})")
    return {"adm_code": None, "adm_name": None, "region_group": None}

In [30]:
def batch_geocode_with_cache(df, api_key, sleep_sec=0.4):
    print(f"📍 전체 좌표 수: {len(df)}")

    unique_coords = df[["lat", "lon"]].drop_duplicates().values.tolist()
    print(f"✅ 고유 좌표 수: {len(unique_coords)}")

    coord_to_adm = {}

    for idx, (lat, lon) in enumerate(tqdm(unique_coords, desc="Kakao API 호출")):
        key = (round(lat, 6), round(lon, 6))
        if key not in coord_to_adm:
            info = get_adm_info_kakao(lat, lon, api_key)
            coord_to_adm[key] = info

            # ▶ 처음 20개까지만 출력
            if idx < 20:
                print(f"[{idx+1}] ({lat:.5f}, {lon:.5f}) → {info}")

            time.sleep(sleep_sec)

    print("✅ 전체 좌표 처리 완료!")
    return coord_to_adm

In [219]:
#전체 df에 결과 매핑
def assign_adm_info(df, coord_to_adm):
    adm_codes, adm_names, region_groups = [], [], []

    for lat, lon in df[["lat", "lon"]].values:
        key = (round(lat, 6), round(lon, 6))
        info = coord_to_adm.get(key, {"adm_code": None, "adm_name": None, "region_group": None})
        adm_codes.append(info["adm_code"])
        adm_names.append(info["adm_name"])
        region_groups.append(info["region_group"])

    df["adm_code"] = adm_codes
    df["adm_name"] = adm_names
    df["region_group"] = region_groups
    return df

In [225]:
API_KEY = "4ffdaedd97065f6660c65fe7c75d42e6"

# STEP 1: 고유 좌표만 호출
coord_to_adm = batch_geocode_with_cache(db_21, API_KEY)

# STEP 2: 전체 df에 결과 적용
# db_21 = assign_adm_info(db_21, coord_to_adm)

📍 전체 좌표 수: 29410
✅ 고유 좌표 수: 29410


Kakao API 호출:   0%|                                 | 0/29410 [00:00<?, ?it/s]

[1] (36.96663, 127.06393) → {'adm_code': '4122025021', 'adm_name': '팽성읍', 'region_group': '경기도'}


Kakao API 호출:   0%|                       | 1/29410 [00:00<3:48:01,  2.15it/s]

[2] (36.96963, 127.06393) → {'adm_code': '4122025027', 'adm_name': '팽성읍', 'region_group': '경기도'}


...

[19] (36.99963, 127.17193) → {'adm_code': '4155025029', 'adm_name': '공도읍', 'region_group': '경기도'}


Kakao API 호출:   0%|                      | 19/29410 [00:08<3:51:05,  2.12it/s]

[20] (36.99963, 127.27093) → {'adm_code': '4155012400', 'adm_name': '도기동', 'region_group': '경기도'}


Kakao API 호출:  17%|███▍                | 5001/29410 [44:48<3:38:43,  1.86it/s]


KeyboardInterrupt: 

In [ ]:
# STEP 1: 고유 좌표만 호출
coord_to_adm = batch_geocode_with_cache(db_26, API_KEY)

# STEP 2: 전체 df에 결과 적용
db_26 = assign_adm_info(db_26, coord_to_adm)

In [ ]:
# STEP 1: 고유 좌표만 호출
coord_to_adm = batch_geocode_with_cache(db_30, API_KEY)

# STEP 2: 전체 df에 결과 적용
db_30 = assign_adm_info(db_30, coord_to_adm)

##### adm_code, adm_name, region_group 채우기 (geojson)

In [15]:
gdf_dong = gpd.read_file("metropolitan_dong.geojson") 

gdf_dong.head()

,adm_code,adm_name,region_group,geometry
0,11110101,청운동,서울특별시,"POLYGON ((126.97556 37.58968, 126.97549 37.589..."
1,11110102,신교동,서울특별시,"POLYGON ((126.97031 37.58418, 126.97033 37.584..."
2,11110103,궁정동,서울특별시,"POLYGON ((126.974 37.58654, 126.97401 37.58653..."
3,11110104,효자동,서울특별시,"POLYGON ((126.97356 37.58323, 126.97355 37.582..."
4,11110105,창성동,서울특별시,"POLYGON ((126.97353 37.58182, 126.97354 37.581..."


In [42]:
# ✅ STEP 1. 모든 포인트 데이터에 행정동 정보 붙이기
def enrich_with_adm_info(df, geojson):
    gdf = gpd.GeoDataFrame(df.copy(), geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")
    joined = gpd.sjoin(gdf, geojson, how="left", predicate="within")
    return pd.concat([
        df.reset_index(drop=True),
        joined[["adm_code", "adm_name", "geometry"]].reset_index(drop=True)
    ], axis=1)

# 적용
df_21_enriched = enrich_with_adm_info(d_s_21_scale, gdf_dong)
df_26_enriched = enrich_with_adm_info(d_s_26_scale, gdf_dong)
df_30_enriched = enrich_with_adm_info(d_s_30_scale, gdf_dong)
ch_enriched = enrich_with_adm_info(ch, gdf_dong)
pk_enriched = enrich_with_adm_info(pk, gdf_dong)

In [54]:
ch_enriched.head()

,충전소명,주소,상세주소,충전기ID,충전방식,충전상태,설치년도,lat,lon,addr_clean,name_clean,owner_type,adm_code,adm_name,geometry
0,낙성대동주민센터,서울특별시 관악구 낙성대로4가길 5,NaN,1.0,단독,사용가능,2017.0,37.510269,127.076397,서울특별시 관악구 낙성대로4가길 5,낙성대동주민센터,공영,11710101,잠실동,POINT (127.0764 37.51027)
1,롯데마트 중계점,서울특별시 노원구 노원로 330,옥상 매장 앞,1.0,단독,사용가능,2017.0,37.459645,127.042625,서울특별시 노원구 노원로 330,롯데마트 중계점,민영,11650104,원지동,POINT (127.04262 37.45964)
2,서울추모공원,서울특별시 서초구 양재대로12길 74,1층 입구,1.0,단독,사용가능,2017.0,37.452033,127.042811,서울특별시 서초구 양재대로12길 74,서울추모공원,공영,11650104,원지동,POINT (127.04281 37.45203)
3,아시아공원 공영주차장,서울특별시 송파구 잠실동 84,우측 끝,1.0,단독,사용가능,2017.0,37.510269,127.076397,서울특별시 송파구 잠실동 84,아시아공원 공영주차장,공영,11710101,잠실동,POINT (127.0764 37.51027)
4,아시아공원 공영주차장,서울특별시 송파구 잠실동 84,우측 끝,2.0,단독,사용가능,2017.0,37.571988,126.984842,서울특별시 송파구 잠실동 84,아시아공원 공영주차장,공영,11110136,인사동,POINT (126.98484 37.57199)


In [44]:
df_30_enriched['이동경로량'].max()

350323.0

In [46]:
# ✅ STEP 2. 행정동별 groupby 집계

# 이동량
move_21 = df_21_enriched.groupby("adm_code")["이동경로량"].sum().reset_index(name="move_count")
move_26 = df_26_enriched.groupby("adm_code")["이동경로량"].sum().reset_index(name="move_count")
move_30 = df_30_enriched.groupby("adm_code")["이동경로량"].sum().reset_index(name="move_count")

# 충전소
charger = ch_enriched.groupby("adm_code").agg({
    "충전소명": "nunique",
    "충전기ID": "count",
    "owner_type": lambda x: (x == "공영").sum()
}).reset_index()

charger.rename(columns={
    "충전소명": "charger_station_count",
    "충전기ID": "charger_total_count",
    "owner_type": "charger_public_count"
}, inplace=True)

charger["charger_private_count"] = charger["charger_total_count"] - charger["charger_public_count"]

# 주차장
parking = pk_enriched.groupby("adm_code").agg({
    "주차장명": "nunique",
    "주차구획수": "sum",
    "주차장구분": lambda x: (x == "공영").sum()
}).reset_index()

parking.rename(columns={
    "주차장명": "parking_lot_count",
    "주차구획수": "parking_space_total",
    "주차장구분": "parking_public_count"
}, inplace=True)

parking["parking_private_count"] = parking["parking_lot_count"] - parking["parking_public_count"]

In [68]:
move_21.head()

,adm_code,move_count
0,11110101,4704.0
1,11110102,7549.0
2,11110103,1284.0
3,11110104,1902.0
4,11110106,1725.0


In [60]:
parking.head()

,adm_code,parking_lot_count,parking_space_total,parking_public_count,parking_private_count
0,11110106,1,25,0,1
1,11110112,2,54,0,2
2,11110114,2,39,1,1
3,11110117,2,31,1,1
4,11110118,1,1260,1,0


In [56]:
# ✅ STEP 3. 병합
df_21_final = move_21.merge(charger, on="adm_code", how="outer")
print(df_21_final.head())
df_21_final = df_21_final.merge(parking, on="adm_code", how="outer")
df_26_final = move_26.merge(charger, on="adm_code", how="outer")
df_26_final = df_26_final.merge(parking, on="adm_code", how="outer")
df_30_final = move_30.merge(charger, on="adm_code", how="outer")
df_30_final = df_30_final.merge(parking, on="adm_code", how="outer")

# ✅ STEP 4. adm_name, 중심 좌표 등 추가
adm_info = gdf_dong[["adm_code", "adm_name", "geometry"]].copy()
adm_info["lon"] = adm_info.geometry.centroid.x
adm_info["lat"] = adm_info.geometry.centroid.y
adm_info["region_group"] = adm_info["adm_code"].astype(str).str[:2].map({
    "11": "서울특별시",
    "28": "인천광역시",
    "41": "경기도"
})

df_21_final = df_21_final.merge(adm_info.drop(columns=["geometry"]), on="adm_code", how="left")
df_26_final = df_26_final.merge(adm_info.drop(columns=["geometry"]), on="adm_code", how="left")
df_30_final = df_30_final.merge(adm_info.drop(columns=["geometry"]), on="adm_code", how="left")

   adm_code  move_count  charger_station_count  charger_total_count  \
0  11110101      4704.0                    NaN                  NaN   
1  11110102      7549.0                    1.0                  1.0   
2  11110103      1284.0                    NaN                  NaN   
3  11110104      1902.0                    NaN                  NaN   
4  11110106      1725.0                    NaN                  NaN   

   charger_public_count  charger_private_count  
0                   NaN                    NaN  
1                   1.0                    0.0  
2                   NaN                    NaN  
3                   NaN                    NaN  
4                   NaN                    NaN  


/var/folders/8h/6gyg3r851n3gfsrdx658sbbc0000gn/T/ipykernel_95161/3895150354.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  adm_info["lon"] = adm_info.geometry.centroid.x
/var/folders/8h/6gyg3r851n3gfsrdx658sbbc0000gn/T/ipykernel_95161/3895150354.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  adm_info["lat"] = adm_info.geometry.centroid.y


In [52]:
df_21_final.head()

,adm_code,move_count,charger_station_count,charger_total_count,charger_public_count,charger_private_count,parking_lot_count,parking_space_total,parking_public_count,parking_private_count,adm_name,lon,lat,region_group
0,11110101,4704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,청운동,126.969324,37.589205,서울특별시
1,11110102,7549.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,신교동,126.967942,37.584506,서울특별시
2,11110103,1284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,궁정동,126.972706,37.584698,서울특별시
3,11110104,1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,효자동,126.971953,37.582668,서울특별시
4,11110106,1725.0,NaN,NaN,NaN,NaN,1.0,25.0,0.0,1.0,통의동,126.972870,37.578077,서울특별시


In [62]:
fill_cols = [
    "charger_station_count", "charger_total_count",
    "charger_public_count", "charger_private_count",
    "parking_lot_count", "parking_space_total",
    "parking_public_count", "parking_private_count"
]

df_21_final[fill_cols] = df_21_final[fill_cols].fillna(0)
df_26_final[fill_cols] = df_26_final[fill_cols].fillna(0)
df_30_final[fill_cols] = df_30_final[fill_cols].fillna(0)

In [74]:
# ✅ STEP 5. 분석 필드 생성
df_21_final["gap"] = df_21_final["move_count"] - df_21_final["charger_total_count"]
df_21_final["gap_signedlog_emph"] = df_21_final["gap"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))

df_26_final["gap"] = df_26_final["move_count"] - df_26_final["charger_total_count"]
df_26_final["gap_signedlog_emph"] = df_26_final["gap"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))

df_30_final["gap"] = df_30_final["move_count"] - df_30_final["charger_total_count"]
df_30_final["gap_signedlog_emph"] = df_30_final["gap"].apply(lambda x: np.sign(x) * (abs(x) ** 1.5))



In [76]:
df_21_final

,adm_code,move_count,charger_station_count,charger_total_count,charger_public_count,charger_private_count,parking_lot_count,parking_space_total,parking_public_count,parking_private_count,adm_name,lon,lat,region_group,gap,gap_signedlog_emph
0,11110101,4704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,청운동,126.969324,37.589205,서울특별시,4704.0,322627.193001
1,11110102,7549.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,신교동,126.967942,37.584506,서울특별시,7548.0,655764.401742
2,11110103,1284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,궁정동,126.972706,37.584698,서울특별시,1284.0,46009.502323
3,11110104,1902.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,효자동,126.971953,37.582668,서울특별시,1902.0,82949.881302
4,11110106,1725.0,0.0,0.0,0.0,0.0,1.0,25.0,0.0,1.0,통의동,126.972870,37.578077,서울특별시,1725.0,71644.630818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,41830370,NaN,7.0,7.0,1.0,6.0,0.0,0.0,0.0,0.0,청운면,127.752207,37.548210,경기도,NaN,NaN
1305,41830380,NaN,18.0,31.0,13.0,18.0,0.0,0.0,0.0,0.0,양동면,127.742214,37.442832,경기도,NaN,NaN
1306,41830395,NaN,9.0,16.0,0.0,16.0,0.0,0.0,0.0,0.0,지평면,127.651431,37.448687,경기도,NaN,NaN
1307,41830400,2312.0,33.0,61.0,4.0,57.0,2.0,284.0,2.0,0.0,용문면,127.588286,37.519753,경기도,2251.0,106798.030183


In [78]:
df_26_final

,adm_code,move_count,charger_station_count,charger_total_count,charger_public_count,charger_private_count,parking_lot_count,parking_space_total,parking_public_count,parking_private_count,adm_name,lon,lat,region_group,gap,gap_signedlog_emph
0,11110101,42312.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,청운동,126.969324,37.589205,서울특별시,42312.0,8.703528e+06
1,11110102,34626.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,신교동,126.967942,37.584506,서울특별시,34625.0,6.442949e+06
2,11110103,12366.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,궁정동,126.972706,37.584698,서울특별시,12366.0,1.375130e+06
3,11110104,17922.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,효자동,126.971953,37.582668,서울특별시,17922.0,2.399273e+06
4,11110106,15585.0,0.0,0.0,0.0,0.0,1.0,25.0,0.0,1.0,통의동,126.972870,37.578077,서울특별시,15585.0,1.945630e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,41830370,256.0,7.0,7.0,1.0,6.0,0.0,0.0,0.0,0.0,청운면,127.752207,37.548210,경기도,249.0,3.929154e+03
1309,41830380,5610.0,18.0,31.0,13.0,18.0,0.0,0.0,0.0,0.0,양동면,127.742214,37.442832,경기도,5579.0,4.167106e+05
1310,41830395,30054.0,9.0,16.0,0.0,16.0,0.0,0.0,0.0,0.0,지평면,127.651431,37.448687,경기도,30038.0,5.206028e+06
1311,41830400,46616.0,33.0,61.0,4.0,57.0,2.0,284.0,2.0,0.0,용문면,127.588286,37.519753,경기도,46555.0,1.004499e+07


In [80]:
df_30_final

,adm_code,move_count,charger_station_count,charger_total_count,charger_public_count,charger_private_count,parking_lot_count,parking_space_total,parking_public_count,parking_private_count,adm_name,lon,lat,region_group,gap,gap_signedlog_emph
0,11110101,71298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,청운동,126.969324,37.589205,서울특별시,71298.0,1.903777e+07
1,11110102,56788.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,신교동,126.967942,37.584506,서울특별시,56787.0,1.353236e+07
2,11110103,20616.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,궁정동,126.972706,37.584698,서울특별시,20616.0,2.960102e+06
3,11110104,29766.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,효자동,126.971953,37.582668,서울특별시,29766.0,5.135476e+06
4,11110106,26112.0,0.0,0.0,0.0,0.0,1.0,25.0,0.0,1.0,통의동,126.972870,37.578077,서울특별시,26112.0,4.219492e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,41830370,631.0,7.0,7.0,1.0,6.0,0.0,0.0,0.0,0.0,청운면,127.752207,37.548210,경기도,624.0,1.558752e+04
1311,41830380,12318.0,18.0,31.0,13.0,18.0,0.0,0.0,0.0,0.0,양동면,127.742214,37.442832,경기도,12287.0,1.361974e+06
1312,41830395,54799.0,9.0,16.0,0.0,16.0,0.0,0.0,0.0,0.0,지평면,127.651431,37.448687,경기도,54783.0,1.282238e+07
1313,41830400,81892.0,33.0,61.0,4.0,57.0,2.0,284.0,2.0,0.0,용문면,127.588286,37.519753,경기도,81831.0,2.340867e+07


In [86]:
df_21_final["highlight"] = True
df_26_final["highlight"] = True
df_30_final["highlight"] = True

In [94]:
ordered_columns = [
    # 📌 행정동 기본 정보
    "adm_code", "adm_name", "region_group","lat", "lon", 
    
    # 🚗 이동량
    "move_count",
    
    # ⚡ 충전소 정보
    "charger_station_count", "charger_total_count",
    "charger_public_count", "charger_private_count",
    
    # 🅿️ 주차장 정보
    "parking_lot_count", "parking_space_total",
    "parking_public_count", "parking_private_count",
    
    # 📊 격차 및 분석
    "gap", "gap_signedlog_emph", "highlight"
]

In [96]:
df_21_final = df_21_final[ordered_columns]
df_26_final = df_26_final[ordered_columns]
df_30_final = df_30_final[ordered_columns]

In [114]:
fill_cols = [
    'move_count',
    'gap',
    'gap_signedlog_emph'
]

df_21_final[fill_cols] = df_21_final[fill_cols].fillna(0)
df_26_final[fill_cols] = df_26_final[fill_cols].fillna(0)
df_30_final[fill_cols] = df_30_final[fill_cols].fillna(0)

In [118]:
df_21_final.to_csv("21년도_final_DB.csv")
df_26_final.to_csv("26년도_final_DB.csv")
df_30_final.to_csv("30년도_final_DB.csv")